## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-13-01-00 +0000,wsj,Durable-Goods Orders Bounce Back on Surge in A...,https://www.wsj.com/economy/durable-goods-orde...
1,2025-09-25-13-00-00 +0000,missionlocal,Beauty and the cookie monster,https://missionlocal.org/2025/09/sf-photos-bea...
2,2025-09-25-13-00-00 +0000,missionlocal,66 buildings in S.F. face foreclosure after Ve...,https://missionlocal.org/2025/09/sf-veritas-fo...
3,2025-09-25-12-58-45 +0000,nypost,White House orders agencies to prep for mass f...,https://nypost.com/2025/09/25/us-news/white-ho...
4,2025-09-25-12-53-12 +0000,nypost,Republican Jack Ciattarelli in dead heat with ...,https://nypost.com/2025/09/25/us-news/jack-cia...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
96,trump,51
297,new,16
195,ice,15
251,dallas,12
253,shooting,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
219,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,96
191,2025-09-24-22-55-47 +0000,nypost,Trump’s pro-Ukraine shift a ‘strategic move’ b...,https://nypost.com/2025/09/24/world-news/trump...,93
26,2025-09-25-11-58-28 +0000,nyt,Trump Assures Arab Leaders He Won’t Allow Isra...,https://www.nytimes.com/2025/09/25/world/middl...,91
29,2025-09-25-11-54-03 +0000,nypost,Colin Firth’s ex-wife Livia Giuggioli tears up...,https://nypost.com/2025/09/25/us-news/livia-gi...,88
320,2025-09-24-15-09-41 +0000,cbc,Health Canada pushes back against Trump's clai...,https://www.cbc.ca/news/politics/health-canada...,85


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
219,96,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com
205,63,2025-09-24-22-20-12 +0000,nypost,"Vance tears into Democrats, calls out Gavin Ne...",https://nypost.com/2025/09/24/us-news/vance-te...
3,55,2025-09-25-12-58-45 +0000,nypost,White House orders agencies to prep for mass f...,https://nypost.com/2025/09/25/us-news/white-ho...
327,47,2025-09-24-14-37-59 +0000,nypost,Here’s what Jimmy Kimmel said — and if he apol...,https://nypost.com/2025/09/24/media/jimmy-kimm...
143,46,2025-09-25-02-51-33 +0000,nypost,"Mom who lost daughter, 15, to suicide after on...",https://nypost.com/2025/09/24/world-news/mom-w...
54,44,2025-09-25-10-49-00 +0000,wsj,Denmark Says New Drone Flights Over Military B...,https://www.wsj.com/world/europe/denmark-says-...
283,36,2025-09-24-18-30-00 +0000,wsj,Israel Shuts Key Border Crossing Between Jorda...,https://www.wsj.com/world/middle-east/israel-s...
321,35,2025-09-24-15-06-31 +0000,bbc,Israeli forces near Gaza City centre as Hamas ...,https://www.bbc.com/news/articles/c4gk9dreldxo...
320,34,2025-09-24-15-09-41 +0000,cbc,Health Canada pushes back against Trump's clai...,https://www.cbc.ca/news/politics/health-canada...
29,33,2025-09-25-11-54-03 +0000,nypost,Colin Firth’s ex-wife Livia Giuggioli tears up...,https://nypost.com/2025/09/25/us-news/livia-gi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
